# Лабораторная работа №3. Линейные модели для классификации

В задачах классификации, в отличие от задач регрессии, требуется по образу объекта определить его принадлежность к тому или иному классу.
В данной работе будет рассмотрена только задача бинарной (двухклассовой) классификации.
В качестве ответа классификатора используется
$$
P(C_1 | \mathbf\phi) = \sigma( \mathbf{w}^T \mathbf\phi),
$$
где
$$
\sigma(a) = \frac{1}{1 + \operatorname{e}^{-a}}.
$$
 
В случае логистической регрессии минимизируется функция штрафа
$$
E(\mathbf{w}) = -\sum_{n=1}^N\left( t_n \ln y_n + (1 - t_n) \ln (1 - y_n) \right).
$$

Её градиент равен
$$
\nabla E(\mathbf{w}) = \sum_{n=1}^N (y_n - t_n) \mathbf\phi(\mathbf{x}_n) =
\sum_{n=1}^N \left( \sigma(\mathbf{w}^T\mathbf\phi_n - t_n \right) \mathbf\phi_n = \mathbf\Phi^T (\mathbf{y} - \mathbf{T}),
$$
где $\mathbf{y} = (y_1, \dots, y_N)^T$ $-$ отклики распознавателя, $\mathbf{T} = (t_1, \dots, t_N)^T$ $-$ метки классов.
 
Выпишем классический метод градиентного спуска в матричной форме:
$$
\mathbf{w}_{i+1} = \mathbf{w}_i - \alpha\nabla E,\;\;\; i = 1, 2, \dots,
$$
где $\mathbf{w}_0$ задано, а $\nabla E = \nabla E(\mathbf{w}_i, T)$.
Заметим, что здесь $T = \left\{ (\mathbf{x}_k, t_k) \right\}_{k=1}^N$.
То есть в итерационной процедуре на каждой итерации для расчета $\nabla{E}$ используются вся выборка целиком.

Будем теперь вместо полной выборки $T = \left\{ (\mathbf{x}_k, t_k) \right\}_{k=1}^N$ на каждой итерации случайно выбирать $K$ элементов из T:
$T_i = \left( (\mathbf{x}_{i_1}, t_{i_1}), (\mathbf{x}_{i_2}, t_{i_2}), \dots, (\mathbf{x}_{i_K}, t_{i_K}) \right)$.
Назовем $T_i$ пакетом или пачкой (batch).
Индекс $i$ отражает то, что пакет векторов из обучающей выборки выбирается отдельно на каждой итерации.

При $K = 1$ получаем так называемый _стохастический градиентный спуск_ (stochastic gradient descent, SGD).

При $K = N$ используется вся выборка, получаем _классический метод градиентного спуска_ (vanilla gradient descent).

В остальных случаях ($1 < K < N$) метод называют _mini-batch gradient descent_.

На практике обычно _mini-batch gradient descent_ сходится быстрее, чем _vanilla gradient descent_.

Вместе со стохастическим градиентным спуском часто применяют момент.
Момент не позволяет резко измениться направлению спуска.
Это позволяет достичь более плавного и быстрого спуска.
В ряде случаев использование момента также позволяет не «свалиться» в локальный минимум.

**Momentum**.
Итерационный поиск осуществляется по следующему алгоритму:
$$
\mathbf{w}_{i+1} = \mathbf{w}_i + \mathbf{v}_i,\;\;\; i = 1, 2, \dots, \\
\mathbf{v}_i = \gamma \mathbf{v}_{i-1} - \alpha \nabla{E} \left( \mathbf{w}_i, (\mathbf{x}_{k_i}, t_{k_i} \right), \\
\mathbf{v}_0 = 0.
$$

Для параметра $\gamma$ обычно выбирается значение 0.9 или близкое к этому.

In [ ]:
BYzKNd9ENEbzdofCaHAg3r34cVfqEmkAXeqXJUkvpyk=k/2D8Q++FCM6xdjebxKaF7YKXryLZMZjlpmrxGVautg=5yZ3J1kRHF70M7Sk79t/LcEHLuKTQUWPQm8LEnutTm5PiukeP28FyNdKBgH3kaHp22JSA5W6y5OtoYr3qkr6yg==mutw8uGy6SdQGs4HZtUxTNxYrZJbrD2atIOlH86vB4g=mutw8uGy6SdQGs4HZtUxTPgFXVGJnDrjZmt8RZbjjVSGeD/KpQz56P+H03IjlNTx4scQryNmA/OTqv80NROTRPb9Btb1rOUrxx+EaVNiYcM=+0AzMkXNBjlGwIOywaqjlQ==IIlwJEtNoBvWm7UJwtoJR3r5BsFA/LDARQCR0cpFqPd1CsA4pes41lvmxshdrjd+

## Задание №1

Разделите выборку на обучающую и тестовую.
Размер тестовой выборки - 20% от размера исходной выборки.
Параметр `n_features` определяется как `3 + mod(<порядковый номер в списке>, 3)`, `n_samples` необходимо взять `600 - 100 * mod(<порядковый номер в списке>, 3)`.

In [ ]:
gc03TdtlhhZ4i9O1ssAWlG1dd55XxFBwDKVrM5VAMTB56kPhY9aaVK6mXwLM/f6UGH03aUjrMPpxAKaoky3k+pq2hi5vuaXASsS54ycF19Mf4zc7oKLsx8JHsmP1b/TCy7MK+QG8G5wdZtM2d5PA/bTyKzS7Qsf2+GENiZmV9Wlw/jnXEu0d4t0C9gqN32cYa1buwDWiSKzerAvbe+H7MD3utKGdxTFhJaZqvuABe11sE0Owodtk6hRrFys7G92v2/aC3pBVYTVvPqkMBe0K/3Vb0RnYc4GYeYEgXKr0/B388qnImGDhHb7bY+d6yv2P

## Задание №2

Реализуйте алгоритм _mini-batch gradient descent_.
В качестве критерия выхода возьмите ограничение по количеству эпох.

In [ ]:
RK2I9QgobsskunWSyLCu3elsHkgGN9vHdhNp/RdbfmY=g4uVgFXajLaS4jIrAHFr71OzKCg+bEqvNFasmxSXqTt0p4g8F5+t1L5Z5rPK02kER51aidIJA6rbbrfIVbM2lc7Tj7iJQoPM/gSOLUORgVk=AJGKG19bQVomIE+6G1Zoyl1vZCpvMaHwa3J3WL19yTs6Z4UQh8GwlrQOTQxM9GGSczjyC879zp8IF323jkhXTA==cPIjWN0QEnZVVn97tZffz2s7IY97uAs2KJV1cYqJRDbkVH1F+PP71Ge3KMdWb1NigfdnK4b/IawYv4S4nL5zRCi81TiDxNxCI9Tb67a9OroFe6IyJpYkDzgLYlJNylxFB5l8tk0WUt+7QeBVBG2SMw==yoE0tztDesjRVgZ7YuCvZiUnCNkvQVrHEiTgtXwHcMA=pXl+AItx8Lp1tIeG67Z6cc2OIB5phWF/+0Vj8OU7xMRjKzdQXNs2hNunMbHl/w4T+0AzMkXNBjlGwIOywaqjlQ==n++BzQvqaMwnDj+ewesNj+vKH9rmjT+kJMHqf4a1ULbB+7ujqjv55Fpn5I31PoWNRs+PImvlyugsLp5XXQZx+zusk9pRr5d1hybJsi8GD4uA95H1ZV6QZezahhW67CVqpvmTfHwvmhmBzeYATwe8tMzTS3oGfncha45mdXiLVSY=+/8EGYJ64h2dB/O9fikQSIjdTWy4lRJ49/YqKXZktplIHPblYQ5g5w3uHtfxDxqL2n55OBA+FelOUE8My/eqVbaZTb5weBPY4pmuLlJonYmhb9ep5WcZHLNhJCzihAtIZdCHKXBQ7uezKfOdsljgRg==+0AzMkXNBjlGwIOywaqjlQ==g7zicURRRXIbodLCCcrfkLMrR7an5NktYASZWxzi0JDXiP34FPtxXLQc3VwDjnbu56x1zIuNC4XiF30Fo5zFz1jOjyDMsYFP+Ri+N7Un3M0=bWgl73jUPBJeQf5rhGVfjdMX4m8mAN+jHHExrOkM3X64g5PT4eJkewQWCp0cfV5Xz/ZLNUn8J8tmcYLsfXLM4A==tWePcQ7vhPjzM540/RBg+/0ABps1QnV9YEcZQvgN2fAEZsF3QJCesK+rI9C+atv4aiw/JAilAp29bViRxyh5XbVKnm5H1/bkPO0P7nbqsbU8hhtrsQ0fgTPosIjb9g7j0BBKbVTFkQStz5xSPY5ozw==uGAfqQqHYeLnhk0OhindoQpR5XH3TlwxK6wMgfLzEH7mJDkWimMf1v/FnRG/CSfPVQyErkxb0oaQmIhKYqe19w==uGAfqQqHYeLnhk0OhindoTOru1YlNuiUtnys8t8LMK/jN+2Ka7A+LyyewymRg080tVE9WwvPO6I5aXoJyYQsH8PRSbmDg52IUS1Z2GALXBMY+jS2/4JK/Sp4dZUGEcsWuGAfqQqHYeLnhk0OhindoVxU4cs4+HR4HybX0vMX9nNx1LfpPch2M2ruVHk6j8fS+0AzMkXNBjlGwIOywaqjlQ==2izz9sSK7cH82yoGaMaeYAKieVSppLAQf4xOm+rPrNGisjJjPj8mvJawSVf1o0Y/sEyo3r8gj/A72T/xw2rL/9kuwHaMPyaG2iBXsyCSBCRTsIWJyBTGLAEMV3B+SHIf2izz9sSK7cH82yoGaMaeYDT8lx25snEuAZeNDuDP2fnSSueCUd23IYVANd3xsX0iCAauzR9Ylm37Y9mJwbVNDzzQCm5Sxl7E9IEDTH8dcDqxn2TaJaXi5u9hr8gkFTj62izz9sSK7cH82yoGaMaeYJTHYGXiw+IYFfs3cY0Ykt1kXzYPmI9cEXAdLSZDzIIuZas1bRyHdWdW/cajk3d7/lKQ8UGSszvfXtXk1wSrl0K62zIAhdaZlRZmcca4pxoZg+RirVUQxnKjMKjpLgQRUA==2izz9sSK7cH82yoGaMaeYJ/rKp+qKovTgeP5rkps6j+ZALXJu/T772sZVe2bJXfNz9IF+hsDZmD1VIvBHa+GGOxP20JISZTmNF9tILQMvE6lagbWqOsJb9jccf6tZQIA0CchWx+r1v1e95aR1DBOYw==+0AzMkXNBjlGwIOywaqjlQ==lBTxZL0Gjc/PU58WspdoUZBRspA8nyarsetZfPKqPDhj8x23/Wytvni/KjgM+F693WyIqsir5mkbVyWKKOwYVsaGgPNXFGlYNElbuXkOv34=B5l8tk0WUt+7QeBVBG2SMw==/ODje2lqlrrZObtExve79D2TTHjJGsrn/w7nhG3TZaPxFqKOpidO9XqSuxkqPMeTge2gvBxdHXved0FL4mRbOr0ZQoJxdcWSY4SbPyzbuO9z9akAS8rDZ08r7minT1VgWRsILr2M2QGn4YPIPxCFCA==+0AzMkXNBjlGwIOywaqjlQ==Pt5CKY+NfmNKCZWmYXf9Arp29ybH/nEmCx99G1HbXhk=g7zicURRRXIbodLCCcrfkLMrR7an5NktYASZWxzi0JDXiP34FPtxXLQc3VwDjnbu56x1zIuNC4XiF30Fo5zFz1jOjyDMsYFP+Ri+N7Un3M0=AE7XMkYUrpxrDe9cWhnloNw7S6qgRLQV+Gi4xHWRFso60XCxqvo6Q2MrtF2wxSQB0vLvXizGKtNLF1BnsqiraA==+0AzMkXNBjlGwIOywaqjlQ==QFsOInNA/f8t075NcM86a+qyZJwQUC90zXw8GHSlRnQ=AE7XMkYUrpxrDe9cWhnloMZatIPsnfCLudDua4pCXVgSzZgJloQAf2k+NAKu/AVWB5l8tk0WUt+7QeBVBG2SMw==GY+cmyR8oSDOHqBFRhUfyEHxX6FLCK2jm2iqE6FCqc8=AE7XMkYUrpxrDe9cWhnloNVQPlN1zN+AprBy1L/P6aNrkSTcsQLNmOkS7OlsqvVl+0AzMkXNBjlGwIOywaqjlQ==jk7FoQSJoO76k8aqQ1Ffrp6sbhfJRSCZr0Adjh0Qq93ydOzT4NOfH0RxXSJPuzyQwABqeatTdYv38tWA1l8gqEJJwf5+NBaAkCYx/YBXJN27q1153+6MPBwfg9sB3vDynGBMVBmSN72T8ruH0bf0nd2g4++7vy1bSPstCWA68EUfTozgSBIDPtgkhoh8xa9ARePQ8z/6B37CUs8PzKiCKA==8q+95v7rkehosfynRlvFASyUggQrYQ9c5E5PQR/r/TWe5+esDUgdgVNMTTvgVUBDTByXBWlmBY7braonv7H3fUfJiLO4pyrjVxQ/DrD+31M8bQgIsKO16gzl79pOIKrIYlMBv5ujqg34LKAqm1t0eg==

### Задание №2.1

Протестируйте написанный алгоритм, подберите параметры градиентного спуска, дающие после обучения классификатор с наилучшей точностью
($\text{точность} = \frac{\text{количество правильно классифицированных векторов}}{\text{количество векторов}}$).
Сравните классический градиентный спуск, стохастический и mini-batch.
Для начала возьмите следующие параметры:

* случайные начальные значения весов (равномерное распределение на отрезке $[−1; 1]$ для каждой компоненты);
* размер шага обучения $\alpha = 10^{-5}$;
* размер пакета _batch size_ = 20;
* максимальное количество эпох 1000.

Приведите в отчет полученную точность и значение функции штрафа на тестовой и обучающей выборках.
Постройте графики для каждого тестируемого размера батча:

* зависимость функции штрафа на обучающей и тестовой выборках от номера эпохи;
* зависимость точности на обучающей и тестовой выборках от номера эпохи;
* зависимость времени выполнения алгоритма от размера батча.

In [ ]:
n0MSOXfZeQGOkObHf5wb0A==+0AzMkXNBjlGwIOywaqjlQ==mHn1UWLB9/nu9/it4ZiNdw==BMkSQyq3ta91Ro7PxB9rwZACl7POV+9KpVRVg5lLDSMfIqs+qZq+OYFnD0j1nFpRmOrvRoA7ez+Vxu6nP9X5KQ==rsApi0RBs6WqEPgjMQxYCT7i97SjhmASQl84nMTBRp1Groa9dKQq4kTHK+pVNP3UpmEMX+ZlxHea63Yyry64rQ==TU4jSJ/q+tjedg3F/gtj9VfQnGU6wzFku2WYcoTpfe5w5hiSYcYdtkIDJmWrxGXZRggPV5Fn6fQK1BSUigVB/o3XCTZJE3LapTa2VB24dVam9rFkzEAeJdX+nV4dUPnceexE6RDH35gDMMcn7HawzA==EXBtiYN1R1Afucd+vxeInEuM7K+cA2c1IeEx1RLnZu/no3BtniA0ozdqVtnIAykSIK8fOFo85am4kRCWLBxpNQ==WWkEtALhXCRmT5xqIsPfL7/101Wuf2cvopY4aX3yWY0=MC2DuQmEGDGDawsXKDILi0/0IrsYvJ9ApdylFZASw672iUlm7aGIrL9MU+xMKX3y19gZdCgD3/5qEC8ZPP1CAMlZBYuJa6X6ok63z140lqae/hNRV/PZb+kJALrTziCKB5l8tk0WUt+7QeBVBG2SMw==e7Kqt7eSWboqP7ERSkdYn+MIYNoSazQozX7U4ATGMTgI1LPBzfqqg5owUN+zRsSX3Fkk0UQ/yujheF8KYawugw==VwuEdLFYSqHA3jwWuITPPXid8bl/bWbMkhkS35BgnBUyVRjuIEW2Fpbqy+hDwjItCoyWAzXHIv6t96uilDk+Yd26cBUo8QZiehP2ZEcBy3k=VwuEdLFYSqHA3jwWuITPPeXU6PBnjQ7OMzPiR13pfQAim+P0JBXNGWYia5PtL6+sJS/woI4BDB3K4E+KQ9C8lsNZtyvB7y3kSUixDxLt4AY=2dgLNAQOdKfXdZWB/9ycAXfHSHxecO1amOyk5x4utfx5sheorkTy+yIm+MLt38Q2PvgjW8V8UBNB51OHoERL2Ekk84MGjj9UDYSeTv7Msd5OyX/GugQ/1Ww6j+Go3WTX5ZgLRlABzMj43A9NC+CyG/tnAhlkhmHmibK1m+++pA8=vgLC0kz43PwD/WYpAcplU1gyO0h2qsbwaYlGvkvgPbPdJoF878r3fPmyL8lHW7LbQDazs8e1jyiBSdZH97VPucxjJIg2E1k46QilgrRpTSVS0kxTczvbMgnTieGYTNx1vgLC0kz43PwD/WYpAcplU7k1LD8m9X3bovrOZLQINUuns624WCvbZGnwVEd3Hzb8IqrVlddZfPGKLL57bAtyc/8igaDhDeJiyA02BlK1K5Zsnk868kPgCtlwJ9TbEqI/IqaY2Sr8cyLlXsb+w7H56dWQO32gju2WBcf9PhkvrBG2OPiUZ5U0nCH8qwKQeJL5t7SjKpMQZ9CAJsqGnsart4WFm624Ie8FObYb81KFXiu530EzWyJ2tjx3/S9VlNm3LEIBhAk2eM1kthJ7xSBbviO91ErBUwTGSbclohhyoWo=3dnCqBPhBAwNJkE8JxWzhhhGlaTjpJ7O5r1Hm3vMJ495HePJ0YKY/Too66bEYjYmbjHiM0Rnfy8qDbgRoq20xA==B5l8tk0WUt+7QeBVBG2SMw==DI9V7QB/c4Q1vRYq8IOw2g==K7513HvnHmJFAb5PJax+uVkS0BScDiRSirm6F2TcHf/BaIzjJicOJEwir4IyZG61kXvfWlH1SMtLvH2LUOze6g==Rz5q8mAEZ8nqFZOg/29JY0tYueJ+Pxz2g0k9+y/SlHrDunVlO7q1kmd8u4h/FeVpbXhIwrvHjHhQCYtLdUwA7G3zWMxKG30gnvR+TR0i4G4=69tKpragKMy5eY8RBsDi5096i4ogwDp7MFrHpalmwvjK0iUq3T8VN5fhxPBsQeyrXBoXx5nNor44kq059bwj30hTzOtQU/xo/+C9bRMDMcl98OiXQYnd+oCRmLfFWZ36

**Вопросы**

1. Зависит ли точность от размера батча? Почему?
1. Зависит ли время работы алгоритма от размера батча? Почему?

**Ответы**

1. ???
1. ???

## Задание №3

Реализуйте метод оптимизации градиентного спуска _momentum_.
Примените наследование от `GradientDescent` и перегрузите метод `_update`, в котором происходят обновления весов модели.

In [ ]:
acOe7Jx+RAat7pwX8l/RLJvEqNONdM88OaqORjPh7VD9hlX6EDZ6SsZt3Sy9Va4ug4uVgFXajLaS4jIrAHFr74yFDo9uevjE9b93l78O5uWgdTORUkvgIPpNrk3zchFwOHBQt9+dypO6SBS3kltwow==TXv5+msZ9csGUjVm5a01By33dmjPP5xa8DOmyJhQ202edmnzInuUC5nxkSGTtHggn14sk8JlsKDFEkDhxAwlo2lMPZwK/Yt1oIwC6U16V/8=FkBsRDE6ZHJ4P5MOpvlw3w==n++BzQvqaMwnDj+ewesNjydloYoPcIZ/0ag2kGd6ug4t2TQizhttBagLGzMlpaXc9VA6l1XUsZcutsX6kXYk9vNZdOl3I5B8E7qerPfgKjh9yTRze4CvBUoRZYEjECoeD1K3nBZGUwDbC36RH4uVag==TXv5+msZ9csGUjVm5a01Bz9NsA3MQU8YmfsCh46x8Qp3eC7puMFk8Mye5MO6+6ZGB5l8tk0WUt+7QeBVBG2SMw==/ODje2lqlrrZObtExve79D2TTHjJGsrn/w7nhG3TZaPxFqKOpidO9XqSuxkqPMeT9VA6l1XUsZcutsX6kXYk9tWaEIWYPfE+J2ACv2swtqkxJQ8xT3QW67feJJuZfK20hEIbY+2K7qH0zvC4E38nOW2NUmaVnt3m5IJTS5YWV2U=mWZEgkvQErCiSBe3sVGh3xMP7NDRvg1Va6Qp9wEWRYs=

### Задание №3.1

Подберите параметры, дающие улучшение сходимости (в большей степени это относится к параметру $\gamma$).  

* Сравните скорость сходимости со скоростью в задании 2.
Для этого постройте графики, на которых сравниваются скорости сходимости и качества модели для **обучающей** выборки.
* Сравните достигнутую точность с точностью в задании 2.
Удалось ли лучше обучить модель?
Чтобы эксперимент был «честным» следует запускать все методы с одними и теми же параметрами и с одними и теми же начальными значениями весов.

In [ ]:
n0MSOXfZeQGOkObHf5wb0A==+0AzMkXNBjlGwIOywaqjlQ==acgLCokagR6aPX359L/uK4OWm6GsFAnHGF7uvqGMAksz9UdU373RWbh/6oWK6tmyMX2a6ZD6uQjpd0lEBH/veQ==1Y1s7P4er4rhaGIYmKMYvu77Neqcnbf7JPFrrLBbW1OttzNo+cPR5tI7d1MwYWd5yq/hv9/9UncxxoewgmwxnA==+0AzMkXNBjlGwIOywaqjlQ==PKz5ruzvx/CPA1fDZ9yPrAK0ixQ+Rud8XSQKrNUW35TvQNQUEoAxqO8aBVC0BVDfvGzf60tplBT0/bCWtSuJm6FJZpaLY61gF4FnsEM0LAtgJ4ZkuwQjIfcebRhKpxXjVq3LoB4zp1X3HHJUaKKpixYFv8GV/ej49NE22Iz26ko=RggPV5Fn6fQK1BSUigVB/o3XCTZJE3LapTa2VB24dVam9rFkzEAeJdX+nV4dUPnceexE6RDH35gDMMcn7HawzA==WWkEtALhXCRmT5xqIsPfL7/101Wuf2cvopY4aX3yWY0=qAsJOgsUiWmGJabyVpUhYB9ZA/nbiM45OoRmT3X2zx/xxbWjuGrltPyeifeFJkoslZ3ORlSsK+ys3OmmRYakbnugFsXmlruBP/3ftNGH6S5qzKKVOC8qLMzqne/Rnn6QB5l8tk0WUt+7QeBVBG2SMw==e7Kqt7eSWboqP7ERSkdYn+MIYNoSazQozX7U4ATGMTgI1LPBzfqqg5owUN+zRsSX3Fkk0UQ/yujheF8KYawugw==VwuEdLFYSqHA3jwWuITPPXid8bl/bWbMkhkS35BgnBUyVRjuIEW2Fpbqy+hDwjItCoyWAzXHIv6t96uilDk+Yd26cBUo8QZiehP2ZEcBy3k=VwuEdLFYSqHA3jwWuITPPeXU6PBnjQ7OMzPiR13pfQAim+P0JBXNGWYia5PtL6+sJS/woI4BDB3K4E+KQ9C8lsNZtyvB7y3kSUixDxLt4AY=VwuEdLFYSqHA3jwWuITPPbTbGFze97tMOnzOjhPcuweXR+oSNy6rafKJ8qg0BHgcDYFy5WW3dgMnH2dbqq7NQiIbWRc3ntVTmpEZ2E9gorg=2dgLNAQOdKfXdZWB/9ycAXfHSHxecO1amOyk5x4utfx5sheorkTy+yIm+MLt38Q2PvgjW8V8UBNB51OHoERL2Ekk84MGjj9UDYSeTv7Msd5OyX/GugQ/1Ww6j+Go3WTX5ZgLRlABzMj43A9NC+CyG/tnAhlkhmHmibK1m+++pA8=vgLC0kz43PwD/WYpAcplU1gyO0h2qsbwaYlGvkvgPbPdJoF878r3fPmyL8lHW7LbQDazs8e1jyiBSdZH97VPucxjJIg2E1k46QilgrRpTSVS0kxTczvbMgnTieGYTNx1vgLC0kz43PwD/WYpAcplU7k1LD8m9X3bovrOZLQINUuns624WCvbZGnwVEd3Hzb8IqrVlddZfPGKLL57bAtyc/8igaDhDeJiyA02BlK1K5Zsnk868kPgCtlwJ9TbEqI/vgLC0kz43PwD/WYpAcplU3vvGYivq7nYEL+TGoHzp6maj8L2JQCPhvaBiT7G/ONDqH6QfJXPS4uAcXHQ77NkyLUtfNxZxZvkExloDKM8ghZaXzJGtJktjuHEZVKwzwfeIqaY2Sr8cyLlXsb+w7H56dWQO32gju2WBcf9PhkvrBG2OPiUZ5U0nCH8qwKQeJL5t7SjKpMQZ9CAJsqGnsart4WFm624Ie8FObYb81KFXiu530EzWyJ2tjx3/S9VlNm3LEIBhAk2eM1kthJ7xSBbviO91ErBUwTGSbclohhyoWo=3dnCqBPhBAwNJkE8JxWzhgNkQN732i033TFzjg2WUlxeRHfYZNxsrQq+TTduyltyA7atWMSHSrRFmjB7ZO36S9nkdz0YQeMwF49+379/9Iw=

**Вопросы**

1. Какой из алгоритмов быстрее достигает итоговой точности?
1. С ростом коэффициента $\gamma$ алгоритм сходится быстрее или медленее?
1. Что происходит при очень большом коэффициенте $\gamma$?

**Ответы**

1. ???
1. ???
1. ...

## Задание №5

Для какой-либо из обученных моделей постройте график, на котором изображено разделение пространства признаков на две области.
Проверьте, что разделяющая поверхность - линейная.

In [ ]:
/BE6oEGq0JuxG4c6RMaLfRoxxv1ENIttCyhOa1iunpwNYJchsB5whH7cLJy2dkZNmKmDNBeDX+J849zSarutmOsypCtXmmePJ8JqBMAad74=tGJW9WKgELWX7q0nu65+UxFlxi4NuOp54VxDqkBukXqRHCsI1S+9laxMvnKMrY+GU5Rxjm1ohdA4V+XTB5J26F3vbR+e8Ctc6NE3UdkD6uM=VFXCsoK9o+BMNLpZdvZyARxGB3mRopXS5GHp1f40wtyZo67NDnsy8Owg5zm2biL+g7KnpRscUySiJm6XgTp85lLuheCQJyC4pfBiveFNb4B0KxYecveg/31nqT6d/K6vaCdHpzCyCPPX+TB7GBwJFTYnFsiknT8AudWg77NOnYg6V9yTMWDpMze5sb1oad+Lp/4ysjqye6M6ZStOxLTOnTpcWZOsJRDX7+cOiSVlm9nV+zfM3TDNTrHbqkIuIRg1sVYN2BzpelBNFTdZOwIeMHMtMD3QjoThwwvGJEIiOksfB92bBMaz36yv8LRoZLnQ/4eT1pguf4fTQYIbh+aQIwi72e2OkSFGXUUrQzYotaE=+0AzMkXNBjlGwIOywaqjlQ==Q2ZZ5wsJzKMpBIMdsLiGvAWc7RgatsXmmmvGur9DH7dygooNWiywttOagXndTmV4mM/e3MNVTGIUyh1nZ12HUQ==Ja6p4QT8OmgDUkTE2+myEM9vD3Er/9eU8ArhEOy/DqQ=zQFw1I7d7eStuv7JMJFiMz/9NzF+aEWiuR0myVbMUoM=

### Задание №5.1

Классы в выборке не являются линейно разделимыми.
С данной проблемой можно бороться, воздействуя какой-то функцией на старые признаки:
$$
\mathbf{\phi}(\mathbf{x}) = \left( \phi_1(x_1, \dots, x_n), \dots, \phi_n(x_1, \dots, x_n) \right)^T.
$$

Покажите, как изменяется разделяющая поверхность, если задавать разную функцию отображения.
Постройте график, на котором изображено разделение пространства признаков и приведите величину достигнутой точности на тестовой выборке.
Попробуйте добиться лучшей точности (необязательно).

In [ ]:
n0MSOXfZeQGOkObHf5wb0A==+0AzMkXNBjlGwIOywaqjlQ==cQsRZK6gryeStckT+sZJ8RxtPJbRMKcmUO+lHHQHv6o=RhrOmFI2mJKUZO0xPrUE7fZPnZJ//7X6+INvFA63GOA=3WyIqsir5mkbVyWKKOwYVjNtFd7y9Af0ArCNqrll4/w=B5l8tk0WUt+7QeBVBG2SMw==n++BzQvqaMwnDj+ewesNj+vKH9rmjT+kJMHqf4a1ULbB+7ujqjv55Fpn5I31PoWNTXv5+msZ9csGUjVm5a01ByPVLMZFsoN77tPcMKHKPiiShHdF9RGPSusTQyXuCCKNl6O6QD1aBcsplPN0BOiku3ND27GilmJWPZUYiJwsX0Q=B5l8tk0WUt+7QeBVBG2SMw==GY+cmyR8oSDOHqBFRhUfyEHxX6FLCK2jm2iqE6FCqc8=AE7XMkYUrpxrDe9cWhnloNa4IEWQ9N7cyFApxBa1Ro1NQx5jcAR8KJntBrQJDoZ7FNZCayFACicYqzxL9lpjRA==+0AzMkXNBjlGwIOywaqjlQ==bbVvrRSEEzTBOU+o+cTIFsOeVav762XlEBgmLP9KR7g+LZKJPou3whbb9MtHgbjN+Rsr3CsCccAnm/C9NzF1Bg==sYiQYSYq2v0ZMBIiSWrs2UInWZP6byGi+8HZipfZvQHa/dVsq6U9yP5gCQN0q1yIDnXsbqjlOiE6jOO4RvnDhoLOUfkJP7g8LwcFwENURpfxwADQGVn29gUnAK4oTpDP8/GFXZHPGe4SImuSXLiFRmZhLdFHmtm3rlF6wKd6ti97Nz0ljAJ8lBfKkXRrxU7gpJsocd1LMwHkPY5GnE0eFw==+0AzMkXNBjlGwIOywaqjlQ==DI9V7QB/c4Q1vRYq8IOw2g==669ngiBgmzjAOh6zdEADegD0HaynXyuTbmdpDaGbA4yXHjQcK/QfqheCD36QUBuwSBf+4DxiR4WgAkPhBAvsK4qISiai4It/ue5VaqRyZSjim7rdyeyg/9MB1bbxh46msVYN2BzpelBNFTdZOwIeMHMtMD3QjoThwwvGJEIiOksfB92bBMaz36yv8LRoZLnQ/4eT1pguf4fTQYIbh+aQIwi72e2OkSFGXUUrQzYotaE=+0AzMkXNBjlGwIOywaqjlQ==Q2ZZ5wsJzKMpBIMdsLiGvAWc7RgatsXmmmvGur9DH7dygooNWiywttOagXndTmV4mM/e3MNVTGIUyh1nZ12HUQ==Ja6p4QT8OmgDUkTE2+myEM9vD3Er/9eU8ArhEOy/DqQ=zQFw1I7d7eStuv7JMJFiM/22q6MVtHZmidnwLIrJhx4=+0AzMkXNBjlGwIOywaqjlQ==qZ1J8enYtHUakHKVcobquluLbcqjelbEGfEwhX3c7fxMsAsHNIxNUb0SI9LmOKqShCOR7E8kTGM+SL0obATdPUuGxjCUy1Yg6P/eDJNmnks=